In [0]:
df = spark.sql("SELECT * FROM wine.normalized_quality")
display(df.head(5))

features,quality,normalized
"Map(vectorType -> dense, length -> 11, values -> List(7.4, 0.7, 0.0, 1.9, 0.076, 11.0, 34.0, 0.9978, 3.51, 0.56, 9.4))",5,"Map(vectorType -> dense, length -> 11, values -> List(4.250195651414429, 3.909310602902498, 0.0, 1.3475864865502305, 1.6147776972451329, 1.0516094578454602, 1.033581535954439, 528.6822705480491, 22.73515364311872, 3.303698770128244, 8.820761896223381))"
"Map(vectorType -> dense, length -> 11, values -> List(7.8, 0.88, 0.0, 2.6, 0.098, 25.0, 67.0, 0.9968, 3.2, 0.68, 9.8))",5,"Map(vectorType -> dense, length -> 11, values -> List(4.4799359568962895, 4.914561900791712, 0.0, 1.8440657184371576, 2.0822133464476718, 2.3900214951033187, 2.036763614969041, 528.1524226120418, 20.72720560056408, 4.01163422087001, 9.19611346627544))"
"Map(vectorType -> dense, length -> 11, values -> List(7.8, 0.76, 0.04, 2.3, 0.092, 15.0, 54.0, 0.997, 3.26, 0.65, 9.8))",5,"Map(vectorType -> dense, length -> 11, values -> List(4.4799359568962895, 4.24439436886557, 0.2053376101022085, 1.6312889047713315, 1.9547308966651609, 1.4340128970619912, 1.6415706747511676, 528.2583921992433, 21.115840705574655, 3.8346503581845686, 9.19611346627544))"
"Map(vectorType -> dense, length -> 11, values -> List(11.2, 0.28, 0.56, 1.9, 0.075, 17.0, 60.0, 0.998, 3.16, 0.58, 9.8))",6,"Map(vectorType -> dense, length -> 11, values -> List(6.432728553492108, 1.5637242411609995, 2.874726541430919, 1.3475864865502305, 1.5935306222813812, 1.6252146166702566, 1.8239674163901862, 528.7882401352505, 20.46811553055703, 3.4216880119185378, 9.19611346627544))"
"Map(vectorType -> dense, length -> 11, values -> List(7.4, 0.7, 0.0, 1.9, 0.076, 11.0, 34.0, 0.9978, 3.51, 0.56, 9.4))",5,"Map(vectorType -> dense, length -> 11, values -> List(4.250195651414429, 3.909310602902498, 0.0, 1.3475864865502305, 1.6147776972451329, 1.0516094578454602, 1.033581535954439, 528.6822705480491, 22.73515364311872, 3.303698770128244, 8.820761896223381))"


In [0]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol='normalized',labelCol='quality')

train, test = df.select("normalized", "quality").randomSplit([0.8,0.2])

print(f"train df size : {train.count()}, test df size: {test.count()}")

model = lr.fit(train)


train df size : 1303, test df size: 296


---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-1986870356273832> in <cell line: 10>()
      8 
      9 model = lr.fit(train)
---> 10 predictions = model.predict(test)
     11 
     12 display(predictions)

/databricks/spark/python/pyspark/ml/wrapper.py in predict(self, value)
    465         Predict label for the given features.
    466         """
--> 467         return self._call_java("predict", value)

/databricks/spark/python/pyspark/ml/wrapper.py in _call_java(self, name, *args)
     70 
     71         java_args = [_py2java(sc, arg) for arg in args]
---> 72         return _java2py(sc, m(*java_args))
     73 
     74     @staticmethod

/databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1319 
   1320         answer = self.gateway_client.send_command(command)
-> 1321         return_value = get_return_value(
   1322        

In [0]:
model_summary = model.summary

print(f"RMSE: {model_summary.rootMeanSquaredError}")
print(f"R2: {model_summary.r2}")

RMSE: 0.6503933845032388
R2: 0.34067319159470066


In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

eval_rl = RegressionEvaluator(labelCol='quality', predictionCol='prediction')
pred = model.transform(test)
rmse = eval_rl.evaluate(pred, {eval_rl.metricName:'rmse'})
r2 = eval_rl.evaluate(pred, {eval_rl.metricName:"r2"})
print(f"Test sample RSME: {ßrmse}")
print(f"Test model R2: {r2}")

Test sample RSME: 0.6280735940090042
Test model R2: 0.43048020626365746
